# TCN Training Only (Clean)

This notebook is for **training only**.
It uses isolated `train_*` variables and a Sharpe-based checkpoint policy.

## 1) Connect to Colab VM and Sync Repo
Run this first.

In [1]:
# Fresh-start cleanup cell (run before importing project modules)
import gc
import shutil
import subprocess
import sys
from pathlib import Path

TRAIN_REPO_URL = "https://github.com/Dave-DKings/tape_tcn_project.git"
TRAIN_REPO_DIR = Path("/content/adaptive_portfolio_rl")

# 1) Sync repo to latest main
if not (TRAIN_REPO_DIR / ".git").exists():
    subprocess.run(["git", "clone", TRAIN_REPO_URL, str(TRAIN_REPO_DIR)], check=True)

subprocess.run(["git", "-C", str(TRAIN_REPO_DIR), "fetch", "origin"], check=True)
subprocess.run(["git", "-C", str(TRAIN_REPO_DIR), "reset", "--hard", "origin/main"], check=True)

# 2) Remove old experiment outputs/checkpoints/cached data
purge_paths = [
    TRAIN_REPO_DIR / "tcn_fusion_results",
    TRAIN_REPO_DIR / "tcn_results",
    TRAIN_REPO_DIR / "tcn_att_results",
    TRAIN_REPO_DIR / "output_logs",
    TRAIN_REPO_DIR / "data" / "phase1_preparation_artifacts",
    TRAIN_REPO_DIR / "data" / "master_features_NORMALIZED.csv",
    TRAIN_REPO_DIR / "data" / "daily_ohlcv_assets.csv",              # forces fresh OHLCV download
    TRAIN_REPO_DIR / "data" / "processed_daily_macro_features.csv",   # forces fresh macro cache build
]

deleted = []
for p in purge_paths:
    if p.is_dir():
        shutil.rmtree(p, ignore_errors=True)
        deleted.append(str(p))
    elif p.is_file():
        p.unlink(missing_ok=True)
        deleted.append(str(p))

# 3) Remove Python/Jupyter cache folders
for cache_dir in TRAIN_REPO_DIR.rglob("__pycache__"):
    shutil.rmtree(cache_dir, ignore_errors=True)
for ckpt_dir in TRAIN_REPO_DIR.rglob(".ipynb_checkpoints"):
    shutil.rmtree(ckpt_dir, ignore_errors=True)

# 4) Clear loaded project modules from kernel memory
for mod in list(sys.modules.keys()):
    if mod.startswith("src.") or mod.startswith("src_"):
        del sys.modules[mod]
gc.collect()

print("✅ Fresh start complete")
print(f"Repo: {TRAIN_REPO_DIR}")
print(f"Deleted paths: {len(deleted)}")
for d in deleted:
    print(" -", d)

✅ Fresh start complete
Repo: /content/adaptive_portfolio_rl
Deleted paths: 0


In [2]:
#from pathlib import Path
import os

root = Path("/content/adaptive_portfolio_rl")
print("Exists:", root.exists())
print("CWD:", os.getcwd())

print("\nTop-level:")
for p in sorted(root.iterdir()):
    kind = "DIR " if p.is_dir() else "FILE"
    print(f" - [{kind}] {p.name}")

# Quick check for outputs/caches you expected to be deleted
targets = [
    "tcn_fusion_results",
    "tcn_results",
    "tcn_att_results",
    "output_logs",
    "data/phase1_preparation_artifacts",
    "data/master_features_NORMALIZED.csv",
    "data/daily_ohlcv_assets.csv",
    "data/processed_daily_macro_features.csv",
]
print("\nTarget paths:")
for t in targets:
    p = root / t
    print(f" - {t}: {'EXISTS' if p.exists() else 'MISSING'}")


Exists: True
CWD: /content

Top-level:
 - [DIR ] .git
 - [FILE] .gitignore
 - [FILE] RL Portfolio Optimization Feature Engineering.md
 - [FILE] RL_Portfolio_Optimization_Feature_Engineering.ipynb
 - [FILE] USAGE_GUIDE_ACTUARIAL.py
 - [FILE] __init__.py
 - [FILE] convert_md_to_ipynb.py
 - [DIR ] data
 - [FILE] data1.zip
 - [FILE] debug_attention_weights.py
 - [DIR ] eval
 - [DIR ] paper
 - [FILE] ra_kl_research_writeup.ipynb
 - [FILE] rcdcc_research_writeup.ipynb
 - [FILE] requirements.txt
 - [FILE] run_tcn_eval.py
 - [DIR ] src
 - [FILE] tcn_architecture_analysis.ipynb
 - [DIR ] tcn_documentation
 - [FILE] tcn_evaluation_only.ipynb
 - [FILE] technical_deep_dive_presentation.ipynb
 - [DIR ] tests
 - [FILE] traditional_portfolio_benchmarks.ipynb
 - [DIR ] training_scripts

Target paths:
 - tcn_fusion_results: MISSING
 - tcn_results: MISSING
 - tcn_att_results: MISSING
 - output_logs: MISSING
 - data/phase1_preparation_artifacts: MISSING
 - data/master_features_NORMALIZED.csv: MISSING
 - 

In [ ]:
#!find /content/adaptive_portfolio_rl -maxdepth 3 | head -n 300

In [3]:
# Install project requirements in Colab VM
#import subprocess, sys
#from pathlib import Path

REPO_DIR = Path("/content/adaptive_portfolio_rl")
REQ_FILE = REPO_DIR / "requirements.txt"

if not REQ_FILE.exists():
    raise FileNotFoundError(f"Missing requirements file: {REQ_FILE}")

print("Using python:", sys.executable)
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "setuptools", "wheel"], check=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-r", str(REQ_FILE)], check=True)

print("✅ Requirements installed")


Using python: /usr/bin/python3
✅ Requirements installed


In [ ]:
import numpy, pandas, tensorflow
print("numpy", numpy.__version__)
print("pandas", pandas.__version__)
print("tensorflow", tensorflow.__version__)

## 2) Imports

In [4]:
import os, sys
from pathlib import Path

REPO_DIR = Path("/content/adaptive_portfolio_rl")

if not REPO_DIR.exists():
    raise FileNotFoundError(f"Repo not found: {REPO_DIR}")

# Set working directory
os.chdir(REPO_DIR)

# Add repo root to Python path
if str(REPO_DIR) not in sys.path:
    sys.path.insert(0, str(REPO_DIR))

print("cwd:", os.getcwd())
print("sys.path[0]:", sys.path[0])

cwd: /content/adaptive_portfolio_rl
sys.path[0]: /content/adaptive_portfolio_rl


In [5]:
from copy import deepcopy
from pathlib import Path

import pandas as pd

from src.config import get_active_config
from src.csv_logger import CSVLogger
from src.notebook_helpers.tcn_phase1 import prepare_phase1_dataset, run_experiment6_tape

## 3) Base Config and Dataset Prep

In [6]:
# ------------------------------------------------------------------
# Feature lock from CORE project pipeline (no metadata/manifest)
# ------------------------------------------------------------------
from src.data_utils import DataProcessor


def build_core_active_feature_columns(cfg):
    # Active list as defined by core config + current selection rules.
    probe = DataProcessor(cfg)
    return list(dict.fromkeys(probe.get_feature_columns("phase1")))


def apply_core_feature_lock(cfg, active_feature_columns):
    # Compute full candidate pool with selection filter temporarily disabled,
    # then enforce active-only by setting disabled_features = full - active.
    probe_cfg = deepcopy(cfg)
    probe_fp = probe_cfg.setdefault("feature_params", {})
    probe_fs = probe_fp.setdefault("feature_selection", {})
    probe_fs["disable_features"] = False
    probe_fs["disabled_features"] = []

    probe = DataProcessor(probe_cfg)
    core_all_cols = list(dict.fromkeys(probe.get_feature_columns("phase1")))

    active_set = set(active_feature_columns)
    disabled = sorted([c for c in core_all_cols if c not in active_set])

    fp = cfg.setdefault("feature_params", {})
    fs = fp.setdefault("feature_selection", {})
    fs["disable_features"] = True
    fs["disabled_features"] = disabled

    return core_all_cols, disabled


In [7]:
TRAIN_RANDOM_SEED = 42

train_config = deepcopy(get_active_config("phase1"))

# Optional: override analysis horizon
# train_config["ANALYSIS_END_DATE"] = "2025-09-01"

# Build active features from core project files/pipeline.
train_active_feature_columns = build_core_active_feature_columns(train_config)
_, train_disabled_features = apply_core_feature_lock(train_config, train_active_feature_columns)

print("✅ Core feature lock applied")
print("   active_feature_columns:", len(train_active_feature_columns))
print("   disabled_features:", len(train_disabled_features))

# Force fresh dataset build and market data re-download
if "train_phase1_data" in globals():
    del train_phase1_data

train_phase1_data = prepare_phase1_dataset(
    train_config,
    force_download=True,
    preparation_artifacts_dir="/content/adaptive_portfolio_rl/data_exports",
)


✅ Core feature lock applied
   active_feature_columns: 50
   disabled_features: 25
📊 Loading raw market data...
   ✅ Raw data shape: (55107, 7)
   ✅ Date range: 2003-09-02 00:00:00 → 2025-08-29 00:00:00

🔧 Computing multi-horizon log returns: [1, 5, 10, 21]
   ✅ Shape after returns: (54897, 11)

📈 Calculating 21-day rolling statistics

🧮 Computing technical indicators

📊 Computing dynamic covariance features

🎯 Adding regime awareness features
   ✅ Master DF shape: (54897, 47)
   ✅ Total features: 47

📊 Integrating fundamental features (if enabled)...
   ✅ Fundamental columns in dataset: 6 (enabled=True)
   🧾 Sample fundamental cols: ['Fundamental_FCFE_Delta', 'Fundamental_Revenue_Delta', 'Fundamental_NCFO_Delta', 'Fundamental_FCFE_Sign', 'Fundamental_Staleness_Days', 'Fundamental_Staleness_Quarters']

📊 Integrating macroeconomic features (if enabled)...
   ✅ Macro features added - 43 columns: ['EFFR_diff', 'EFFR_zscore', 'SOFR_level', 'SOFR_diff', 'FEDFUNDS_diff', 'FEDFUNDS_zscore', '


💾 Saving NORMALISED master dataframe to '/content/adaptive_portfolio_rl/data/master_features_NORMALIZED.csv'

💾 Saved preparation artifacts:
   raw OHLCV: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260222_061829_raw_ohlcv.csv
   full engineered: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260222_061829_feature_engineered_full.csv
   analysis-window engineered: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260222_061829_feature_engineered_analysis_window.csv
   normalized master: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260222_061829_feature_engineered_normalized.csv
   train normalized: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260222_061829_train_normalized.csv
   test normalized: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260222_061829_test_normalized.csv
   scalers: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260222_061829_scalers.joblib
   audit report: /content/adaptive_portfoli

In [8]:
print("Train shape:", train_phase1_data.train_df.shape)
print("Test shape:", train_phase1_data.test_df.shape)

cols = train_phase1_data.train_df.columns
print("Total columns:", len(cols))

# quick sanity for common redundant groups
dup_like = [c for c in cols if c.endswith("_raw") or c.endswith("_unscaled")]
print("Potential redundant raw/unscaled cols:", len(dup_like))
print(dup_like[:20])

Train shape: (43867, 105)
Test shape: (11030, 105)
Total columns: 105
Potential redundant raw/unscaled cols: 0
[]


In [ ]:
cols

In [9]:
used = set(train_phase1_data.data_processor.get_feature_columns("phase1"))
disabled = set(train_config["feature_params"]["feature_selection"]["disabled_features"])

print("Used feature count:", len(used))
print("Disabled that still in used:", sorted(disabled & used))  # should be []
print("VIX_zscore used?", "VIX_zscore" in used)

Used feature count: 73
Disabled that still in used: []
VIX_zscore used? False


In [10]:
base_cols = ["Date", "Ticker", "Open", "High", "Low", "Close", "Volume"]
keep = [c for c in base_cols + list(used) if c in train_phase1_data.master_df.columns]

train_phase1_data.master_df = train_phase1_data.master_df[keep].copy()
train_phase1_data.train_df = train_phase1_data.train_df[keep].copy()
train_phase1_data.test_df  = train_phase1_data.test_df[keep].copy()

## 4) Training Overrides (Sharpe-Only Checkpoint Policy)

This policy keeps only Sharpe-threshold high-watermark checkpointing (`>= 0.5`) and disables rare/step/periodic/TAPE checkpoint routes.

In [11]:
from copy import deepcopy

train_config = deepcopy(train_config)  # or deepcopy(config) if that's your active object

tp = train_config["training_params"]
ap = train_config["agent_params"]
ppo = ap["ppo_params"]
env = train_config["environment_params"]

# -----------------------------
# 1) Core run / rollout shape
# -----------------------------
tp["max_total_timesteps"] = 100_000
tp["timesteps_per_ppo_update"] = 384  # fallback
tp["timesteps_per_ppo_update_schedule"] = [
    {"threshold": 0, "timesteps_per_update": 384},
    {"threshold": 50_000, "timesteps_per_update": 448},  # gentler than jumping to 512
]
tp["batch_size_ppo_schedule"] = [
    {"threshold": 0, "batch_size": 96},
    {"threshold": 50_000, "batch_size": 112},
]

# -----------------------------
# 2) KL / PPO stability
# -----------------------------
ppo["num_ppo_epochs"] = 2
ppo["policy_clip"] = 0.08
ppo["target_kl"] = 0.022 #0.018
ppo["kl_stop_multiplier"] = 1.25
ppo["minibatches_before_kl_stop"] = 2
ppo["max_grad_norm"] = 0.30

ppo["actor_lr"] = 8e-6
ppo["critic_lr"] = 1.2e-4
tp["actor_lr_schedule"] = [
    {"threshold": 0, "lr": 8e-6},
    {"threshold": 30_000, "lr": 7e-6},
    {"threshold": 60_000, "lr": 6e-6},
]

ppo["entropy_coef"] = 0.0020

# -----------------------------
# 3) Dirichlet controls
# -----------------------------
ap["dirichlet_alpha_activation"] = "elu"
ap["dirichlet_logit_temperature"] = 1.15
ap["dirichlet_alpha_cap"] = 35.0
ap["dirichlet_epsilon"] = {"max": 1.0, "min": 0.35}

# -----------------------------
# 4) Turnover (loosened a bit)
# -----------------------------
env["target_turnover"] = 0.35
env["turnover_penalty_scalar"] = 1.20
env["transaction_cost_pct"] = 0.0008

tp["turnover_penalty_curriculum"] = {
    0: 1.20,
    10_000: 1.50,
    25_000: 1.90,
    40_000: 2.20,
}
tp["evaluation_turnover_penalty_scalar"] = 2.20

# -----------------------------
# 5) Anti-concentration (important)
# -----------------------------
env["concentration_penalty_scalar"] = 2.5
env["concentration_target_hhi"] = 0.12
env["top_weight_penalty_scalar"] = 2.0
env["action_realization_penalty_scalar"] = 0.5

# -----------------------------
# 6) Horizon curriculum (capped)
# -----------------------------
tp["use_episode_length_curriculum"] = True
tp["episode_length_curriculum_schedule"] = [
    {"threshold": 0, "limit": 252},
    {"threshold": 10_000, "limit": 504},
    {"threshold": 25_000, "limit": 756},
    {"threshold": 90_000, "limit": 1008},  # last ~10% cap
]

# -----------------------------
# 7) Logging + checkpoint policy
# -----------------------------
tp["log_step_diagnostics"] = True
tp["alpha_diversity_log_interval"] = 1
tp["alpha_diversity_warning_after_updates"] = 120
tp["alpha_diversity_warning_std_threshold"] = 0.25

tp["high_watermark_checkpoint_enabled"] = True
tp["high_watermark_sharpe_threshold"] = 0.5
tp["step_sharpe_checkpoint_enabled"] = False
tp["periodic_checkpoint_every_steps"] = 0
tp["rare_checkpoint_params"] = {"enable": False}
tp["tape_checkpoint_threshold"] = 999.0

print("✅ Applied KL+diversification+soft-turnover tuning")
print("target_kl:", ppo["target_kl"], "| kl_stop_multiplier:", ppo["kl_stop_multiplier"])
print("turnover curriculum:", tp["turnover_penalty_curriculum"])
print("concentration penalty:", env["concentration_penalty_scalar"], "target_hhi:", env["concentration_target_hhi"])
print("rollout schedule:", tp["timesteps_per_ppo_update_schedule"])
print("horizon schedule:", tp["episode_length_curriculum_schedule"])

✅ Applied KL+diversification+soft-turnover tuning
target_kl: 0.022 | kl_stop_multiplier: 1.25
turnover curriculum: {0: 1.2, 10000: 1.5, 25000: 1.9, 40000: 2.2}
concentration penalty: 2.5 target_hhi: 0.12
rollout schedule: [{'threshold': 0, 'timesteps_per_update': 384}, {'threshold': 50000, 'timesteps_per_update': 448}]
horizon schedule: [{'threshold': 0, 'limit': 252}, {'threshold': 10000, 'limit': 504}, {'threshold': 25000, 'limit': 756}, {'threshold': 90000, 'limit': 1008}]


In [12]:
tp = train_config["training_params"]

# OFF (baseline)
tp["ra_kl_enabled"] = False

# ON (recommended starter)
tp["ra_kl_enabled"] = True
tp["ra_kl_target_ratio"] = 1.0
tp["ra_kl_ema_alpha"] = 0.25
tp["ra_kl_gain"] = 0.06
tp["ra_kl_deadband"] = 0.10
tp["ra_kl_max_change_fraction"] = 0.10
tp["ra_kl_min_target_kl"] = 0.010
tp["ra_kl_max_target_kl"] = 0.035


tp["log_step_diagnostics"] = True  

## 5) Run Training

In [13]:
RUN_TRAINING = True

if RUN_TRAINING:
    tp = train_config["training_params"]
    print("🚀 Starting training")
    print("Architecture:", train_config["agent_params"].get("actor_critic_type"))
    print("max_total_timesteps:", tp["max_total_timesteps"])

    train_experiment6 = run_experiment6_tape(
        phase1_data=train_phase1_data,
        config=train_config,
        random_seed=TRAIN_RANDOM_SEED,
        csv_logger_cls=CSVLogger,
        use_covariance=True,
        architecture=train_config["agent_params"].get("actor_critic_type"),
        timesteps_per_update=tp.get("timesteps_per_ppo_update", 384),
        max_total_timesteps=tp["max_total_timesteps"],
    )

    print("✅ Training complete")
    print("checkpoint_prefix:", train_experiment6.checkpoint_path)
else:
    print("ℹ️ RUN_TRAINING=False")

🚀 Starting training
Architecture: TCN_FUSION
max_total_timesteps: 100000

EXPERIMENT 6: TCN_FUSION Enhanced + TAPE Three-Component
Architecture: TCN + Fusion
Results root: /content/adaptive_portfolio_rl/tcn_fusion_results
Working dir: /content/adaptive_portfolio_rl
Covariance Features: Yes
🎯 REWARD SYSTEM: TAPE (Three-Component v3)
   Profile: BalancedGrowth
   Daily: Base + DSR/PBRS + Turnover_Proximity
   Terminal: mode=signed | baseline=0.20 | scalar=10.0 (clipped ±10.0)
   Gate A: enabled (Sharpe ≤ 0.00 or MDD ≥ 25.0% -> force non-positive terminal bonus)
   Neutral Band: enabled (±0.020 around baseline)
   🔄 Profile Manager: disabled (static profile only)
🎲 Experiment Seed: 6042 (Base: 42, Offset: 6000)
✅ Features: Enhanced (includes 3 covariance eigenvalues)
   Eigenvalues: ['Covariance_Eigenvalue_0', 'Covariance_Eigenvalue_1', 'Covariance_Eigenvalue_2']
   Train shape: (43867, 80)
   Test shape: (11030, 80)

🏗️ Creating THREE-COMPONENT TAPE v3 environments (with curriculum)...
 

   🎯 Episode 2: TAPE Score = 0.2429 (bonus: +0.54 → +0.54)
   🎯 Episode 3: TAPE Score = 0.4932 (bonus: +3.66 → +3.66)
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00003_shp0p951_actor.weights.h5 (Sharpe=0.951)
   🎯 Episode 4: TAPE Score = 0.1928 (bonus: -0.00 → -0.00)
      🟰 Neutral band applied (±0.020)
      🚦 Gate A applied: Sharpe=0.173, MDD=29.04%


   🎯 Episode 5: TAPE Score = 0.4654 (bonus: +3.32 → +3.32)
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00005_shp0p855_actor.weights.h5 (Sharpe=0.855)
   🎯 Episode 6: TAPE Score = 0.5779 (bonus: +4.72 → +4.72)
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00006_shp1p324_actor.weights.h5 (Sharpe=1.324)


   🎯 Episode 7: TAPE Score = 0.2284 (bonus: -0.36 → -0.36)
      🚦 Gate A applied: Sharpe=-0.324, MDD=11.81%


   🎯 Episode 8: TAPE Score = 0.1845 (bonus: -0.00 → -0.00)
      🟰 Neutral band applied (±0.020)
      🚦 Gate A applied: Sharpe=-1.166, MDD=47.62%
   🎯 Episode 9: TAPE Score = 0.2790 (bonus: +0.99 → +0.99)


...

   🎯 Episode 164: TAPE Score = 0.1860 (bonus: -0.00 → -0.00)
      🟰 Neutral band applied (±0.020)
      🚦 Gate A applied: Sharpe=-0.057, MDD=35.38%


   🎯 Episode 165: TAPE Score = 0.2183 (bonus: +0.00 → +0.00)
      🟰 Neutral band applied (±0.020)


🔄 Update 240/242 | Step 99,136/100,000 | Episode 165 | Time: 14318.1s
   📊 Metrics: Return=+11.47% | Sharpe=0.124 | DD=17.79% | Turnover=64.30%
   🎚️ Intra-Step TAPE: potential=0.6114 | delta_reward=-0.0008
   🎯 Profile: BalancedGrowth
   🧠 Training: actor_loss=0.0444 | critic_loss=0.1996 | mean_adv=0.0000
   🧮 Loss Detail: critic_scaled=0.0998 | risk_aux_total=0.0000 | sharpe_proxy=0.0000 | sharpe_loss=0.0000 | mvo_loss=0.0000
   ⚙️ Optimizer: actor_lr=0.000006 | critic_lr=0.000120 | target_kl=0.0100 | rollout=448 | batch_size=112
   🧭 RA-KL: ema_kl=0.03706 | error_ratio=+2.706 | adjust=0.900 | stop_threshold=0.01250
   🔬 Alpha Diversity: mean=2.26 | std=0.48 | range=[1.41, 3.05]
   🔒 Drawdown λ snapshot=0.000 (peak 0.000, dd 0.90% / trig 16.50%) | terminal=0.000 (peak 0.141) | TAPE=0.2183


   🎯 Episode 166: TAPE Score = 0.2231 (bonus: +0.29 → +0.29)


🔄 Update 242/242 | Step 100,000/100,000 | Episode 166 | Time: 14442.3s
   📊 Metrics: Return=+11.74% | Sharpe=0.126 | DD=19.12% | Turnover=64.84%
   🎚️ Intra-Step TAPE: potential=0.3854 | delta_reward=+0.0006
   🎯 Profile: BalancedGrowth
   🧠 Training: actor_loss=-0.0540 | critic_loss=0.4411 | mean_adv=-0.0000
   🧮 Loss Detail: critic_scaled=0.2205 | risk_aux_total=0.0000 | sharpe_proxy=0.0000 | sharpe_loss=0.0000 | mvo_loss=0.0000
   ⚙️ Optimizer: actor_lr=0.000006 | critic_lr=0.000120 | target_kl=0.0100 | rollout=448 | batch_size=112
   🧭 RA-KL: ema_kl=0.02731 | error_ratio=+1.731 | adjust=0.901 | stop_threshold=0.01250
   🔬 Alpha Diversity: mean=2.22 | std=0.51 | range=[1.19, 3.09]
   🔒 Drawdown λ snapshot=0.000 (peak 0.000, dd 2.28% / trig 16.50%) | terminal=0.000 (peak 0.013) | TAPE=0.2231

✅ THREE-COMPONENT TAPE v3 training completed!
   Total episodes: 166
   Total timesteps: 100,000
   Training time: 14442.33s (240.71min)
📊 Training summary saved: /content/adaptive_portfolio_rl/

## 6) Inspect Latest Training Logs

In [14]:
TRAIN_RESULTS_ROOT = Path("/content/adaptive_portfolio_rl/tcn_fusion_results")
TRAIN_LOGS_DIR = TRAIN_RESULTS_ROOT / "logs"

episodes_files = sorted(TRAIN_LOGS_DIR.glob("*episodes*.csv"), key=lambda p: p.stat().st_mtime, reverse=True)
if not episodes_files:
    print(f"No episodes CSV found in {TRAIN_LOGS_DIR}")
else:
    train_episodes_path = episodes_files[0]
    train_episodes_df = pd.read_csv(train_episodes_path)
    print("Episodes file:", train_episodes_path)
    print("Rows:", len(train_episodes_df))
    display(train_episodes_df.tail(20))

Episodes file: /content/adaptive_portfolio_rl/tcn_fusion_results/logs/Exp6_TCN_FUSION_Enhanced_TAPE_training_20260222_062113_episodes.csv
Rows: 13


,update,timestep,episode,elapsed_time,episode_return_pct,episode_sharpe,episode_sortino,episode_max_dd,episode_volatility,episode_win_rate,...,actor_grad_norm,critic_grad_norm,alpha_min,alpha_max,alpha_mean,ratio_mean,ratio_std,drawdown_lambda_peak,episode_length,termination_reason
0,20,7680,30,1145.921034,6.541556,0.296814,0.345831,31.773409,0.312617,54.980080,...,1.894796,0.926265,1.081004,3.646165,2.262659,1.002504,0.206235,0.536717,252.0,episode_limit
1,40,15360,51,2258.782684,5.782132,0.129923,0.186684,11.931834,0.116451,49.701789,...,1.456817,0.254633,1.376851,3.146779,2.273509,1.006397,0.164183,0.063632,504.0,episode_limit
2,60,23040,66,3363.837883,-15.453382,-0.172934,-0.238282,37.301845,0.315964,52.286282,...,2.076430,1.046876,1.204040,3.329773,2.246355,1.017035,0.180322,1.465897,504.0,episode_limit
3,80,30720,77,4470.679057,-5.077024,-0.164928,-0.223215,16.262068,0.153780,52.052980,...,1.843398,1.083311,1.307567,3.238561,2.235099,1.007644,0.166983,0.000000,756.0,episode_limit
4,100,38400,87,5575.839055,-2.186340,-0.150932,-0.228581,20.169731,0.126878,52.185430,...,1.412924,0.569396,1.242966,3.199548,2.228592,1.004902,0.171973,0.035843,756.0,episode_limit
5,120,46080,98,6692.610933,-34.032756,-0.664596,-0.862933,44.852759,0.206770,49.933775,...,2.965063,1.207120,0.711967,3.966176,2.075877,1.020289,0.727280,1.280505,756.0,episode_limit
6,140,54336,108,7897.208880,9.417936,0.145911,0.191812,12.601162,0.117673,49.933775,...,1.365512,0.451789,1.046644,3.140608,2.059454,0.992763,0.172952,0.000000,756.0,episode_limit
7,160,63296,120,9185.661492,2.207604,-0.050664,-0.066522,17.336867,0.115451,50.860927,...,1.651380,0.243322,1.149544,3.109914,2.078926,1.000255,0.180749,0.000837,756.0,episode_limit
8,180,72256,132,10471.297643,6.504814,0.078591,0.100861,18.612846,0.139238,51.655629,...,1.988872,0.224274,1.046454,3.156558,2.082150,1.000443,0.240029,0.003053,756.0,episode_limit
9,200,81216,144,11743.819913,25.649109,0.543708,0.826669,15.613754,0.116102,52.980132,...,2.266486,0.117776,1.132757,3.103621,2.109309,0.986040,0.186952,0.000000,756.0,episode_limit


In [ ]:
train_episodes_df.columns

## 7) Export Results Folder (Optional)
Creates a zip for download from Colab VM.

In [15]:
from pathlib import Path
import subprocess

EXPORT_RESULTS_ZIP = True
EXPORT_PATH = Path("/content/tcn_fusion_results_export2.zip")
ROOT = Path("/content/adaptive_portfolio_rl")

if EXPORT_RESULTS_ZIP:
    include_paths = [
        ROOT / "tcn_fusion_results",
        ROOT / "data" / "phase1_preparation_artifacts",
        ROOT / "data" / "master_features_NORMALIZED.csv",
    ]

    existing = [p for p in include_paths if p.exists()]
    if not existing:
        print("⚠️ Nothing to export.")
    else:
        if EXPORT_PATH.exists():
            EXPORT_PATH.unlink()

        rel_items = [str(p.relative_to(ROOT)) for p in existing]
        cmd = f"cd {ROOT} && zip -qr {EXPORT_PATH} " + " ".join(rel_items)
        subprocess.run(cmd, shell=True, check=True)

        print(f"✅ Created: {EXPORT_PATH}")
        print("Included:")
        for p in rel_items:
            print(" -", p)
else:
    print("ℹ️ EXPORT_RESULTS_ZIP=False")


✅ Created: /content/tcn_fusion_results_export2.zip
Included:
 - tcn_fusion_results
 - data/master_features_NORMALIZED.csv


In [16]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/tcn_fusion_results_export2.zip /content/drive/MyDrive/
print("✅ Copied to Drive: /content/drive/MyDrive/tcn_fusion_results_export2.zip")

Mounted at /content/drive
✅ Copied to Drive: /content/drive/MyDrive/tcn_fusion_results_export2.zip
